In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=8ba3403e48c1a49154316622084b6ddaa7aa843eb883479b68a0f9cc7910d87e
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 18 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected packag

In [ ]:
# Librerias
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
# Carga de datos
from google.colab import drive
drive.mount("/content/drive")
file_path = "/content/drive/MyDrive/TP 1 - ORGANIZACIÓN DE DATOS/googleplaystore.csv"
reviews_file_path = "/content/drive/MyDrive/TP 1 - ORGANIZACIÓN DE DATOS/googleplaystore_user_reviews.csv"


Mounted at /content/drive


In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
google_play_data = sqlContext.read.option("delimiter", ",").option("escape", '"').csv(file_path, header=True, inferSchema=True)
user_reviews_data = sqlContext.read.option("delimiter", ",").option("escape", '"').csv(reviews_file_path, header=True, inferSchema=True)
google_play_data = google_play_data.rdd
user_reviews_data = user_reviews_data.rdd
# Filtra las filas en las que la categoría no sea "1.9" y crea un nuevo RDD sin esa fila
google_play_data = google_play_data.filter(lambda x: x['Category'] != "1.9")


/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


# 9 - Cual es la app más cara de cada categoría (⭐)

Filtrar las filas en las que el tipo no sea 'Free'

In [ ]:
filtered_rdd = google_play_data.filter(lambda x: x['Type'] != "Free")

Mapear cada aplicación con su categoría como clave

In [ ]:
key_value_rdd = filtered_rdd.map(lambda x: (x['Category'], x))

Definir una función para encontrar la aplicación con el precio máximo

In [ ]:
def max_price_app(app1, app2):
    price1 = float(app1['Price'].replace('$', ''))
    price2 = float(app2['Price'].replace('$', ''))
    return app1 if price1 >= price2 else app2

Usar reduceByKey para encontrar la aplicación más cara en cada categoría y map para darle formato al RDD que muestra el resultado

In [ ]:
result_rdd = key_value_rdd.reduceByKey(max_price_app).map(lambda x: (x[0], (x[1]['App'], x[1]['Price'])))

In [ ]:
result_rdd.collect()

[('BUSINESS', ('Lean EQ', '$89.99')),
 ('COMMUNICATION', ('Z PIVOT', '$19.99')),
 ('DATING', ('AMBW Dating App: Asian Men Black Women Interracial', '$7.99')),
 ('EDUCATION', ('Fuzzy Numbers: Pre-K Number Foundation', '$5.99')),
 ('ENTERTAINMENT', ('My Talking Pet', '$4.99')),
 ('FOOD_AND_DRINK', ('Paprika Recipe Manager', '$4.99')),
 ('HEALTH_AND_FITNESS', ('Vital Tones EI Pro', '$9.99')),
 ('GAME', ('The World Ends With You', '$17.99')),
 ('FAMILY', ('most expensive app (H)', '$399.99')),
 ('MEDICAL', ('EP Cook Book', '$200.00')),
 ('PHOTOGRAPHY', ('Guide to Nikon Df', '$29.99')),
 ('SPORTS', ('Golfshot Plus: Golf GPS', '$29.99')),
 ('PERSONALIZATION', ('BIG Launcher', '$9.99')),
 ('PRODUCTIVITY', ('cronometra-br', '$154.99')),
 ('WEATHER', ('Florida Tides & Weather', '$6.99')),
 ('TOOLS', ('G-NetReport Pro', '$25.99')),
 ('TRAVEL_AND_LOCAL', ('Maps & GPS Navigation OsmAnd+', '$8.99')),
 ('LIFESTYLE', ("I'm Rich - Trump Edition", '$400.00')),
 ('AUTO_AND_VEHICLES', ('FORD V SERIES CAL